# HW1 Part 2A

README:

Just run all the code cells. Sorry for that some codes may not fully show in the pdf due to the cell length. But you could COPY AND PASTE them somewhere to see the full codes.

In [2]:
# import necessary libraries
library(caret) # for train, predict, etc.
library(doParallel) # for parallel computing
library(klaR) # for "train"
library(quanteda) # for "textmodel_nb"
library(OpenImageR) # for "resizeImage"

In [3]:
# parallel computation
registerDoParallel(makeCluster(detectCores()))

In [4]:
# ALL INPUT HERE
dataTrain <- read.csv('train.csv')
dataTest <- read.csv('test.csv', header = F)
dataVal <- read.csv('val.csv')

In [5]:
# spilt features and labels in training data and validation data
train.feature <- dataTrain[,3:786]
train.label <- as.factor(dataTrain[,2])
val.feature <- dataVal[,2:785] 
val.label <- as.factor(dataVal[,1])

## MODEL1: Gaussian & Untouched

In [6]:
# model training
model1 <- train(x = train.feature , y = train.label, method = 'naive_bayes', trControl=trainControl(method='cv', number=10))

In [7]:
# model prediction
val.pred1 <- predict(model1, newdata = val.feature)
confusionMatrix(data = val.pred1, val.label)

Confusion Matrix and Statistics

          Reference
Prediction   0   1   2   3   4   5   6   7   8   9
         0 158   0  36  26   8  27   1   1   5   4
         1   0 242   5  19   3   5   7   5  39   3
         2   0   0  31   1   1   0   0   0   3   0
         3   0   0  11  67   0   4   0   1   2   1
         4   0   0   0   0  30   0   1   0   1   0
         5   0   0   1   0   2  10   0   0   1   0
         6   7   3  71  16  26  19 188   3  15   4
         7   0   0   0   0   4   0   0  68   1   3
         8   0   2  25  34   9  72   2   3  85   2
         9   3   2   5  47 112  30   0 147  52 178

Overall Statistics
                                          
               Accuracy : 0.5285          
                 95% CI : (0.5063, 0.5506)
    No Information Rate : 0.1245          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4751          
 Mcnemar's Test P-Value : NA              

Statistics by Class:


Accuracy : 0.5285

## MODEL2: Bernouli & Untouched

In [8]:
# model training
model2 <- textmodel_nb(as.dfm(train.feature), as.factor(train.label), distribution = 'Bernoulli')

In [9]:
# model prediction
val.pred2 <- predict(model2, newdata = as.dfm(val.feature))
confusionMatrix(data = val.pred2, val.label)

Confusion Matrix and Statistics

          Reference
Prediction   0   1   2   3   4   5   6   7   8   9
         0 153   0   4   1   0   3   0   0   3   1
         1   0 241   3   4   3   1   3   7   6   0
         2   1   1 154   8   1   3   6   4   6   0
         3   1   0   6 172   0  25   0   0  16   8
         4   1   0   2   1 150   5   2   7   3  17
         5   5   4   0   5   1 118   8   1   4   1
         6   3   1   8   0   7   3 180   0   1   1
         7   0   0   3   5   2   1   0 191   1  12
         8   4   2   4   5   4   5   0   5 157   4
         9   0   0   1   9  27   3   0  13   7 151

Overall Statistics
                                          
               Accuracy : 0.8335          
                 95% CI : (0.8164, 0.8496)
    No Information Rate : 0.1245          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8146          
 Mcnemar's Test P-Value : NA              

Statistics by Class:


Accuracy : 0.8335

#### Stretched Bounding Function

In [10]:
bounding <- function(dataIN){
    cubic <- matrix(,nrow(dataIN),400) 
    for (i in 1:nrow(dataIN)){
        cubic28 <- matrix(dataIN[i, ],28,28)
        for (j in 1:28){
            if (Reduce("+",cubic28[j,])>0){
                lowerbound <- j
                break
            }
        }
        for (j in 28:1){
            if (Reduce("+",cubic28[j,])>0){
                upperbound <- j
                break
            }
        }
        for (j in 1:28){
            if (Reduce("+",cubic28[,j])>0){
                leftbound <- j
                break
            }
        }
        for (j in 28:1){
            if (Reduce("+",cubic28[,j])>0){
                rightbound <- j
                break
            }
        }
        cubic20 <- cubic28[lowerbound:upperbound, leftbound:rightbound]
        cubic20 <- apply(resizeImage(as.matrix.data.frame(cubic20), 20, 20, method = "bilinear"), 1, function(x)x*255)
        cubic[i, ] <- cubic20
    }
    return (cubic)
}

## MODEL3: Gaussian & Stretched Bounding

In [11]:
# data reprocessing
train.cubic = bounding(train.feature)
val.cubic = bounding(val.feature)
colnames(train.cubic) <- colnames(train.feature)[1:400]
colnames(val.cubic) <- colnames(train.feature)[1:400]

In [12]:
# model training
model3 <- train(x = train.cubic, y = train.label, method = 'naive_bayes', trControl=trainControl(method='cv', number=10))

In [13]:
# model prediction
val.pred3 <- predict(model3, newdata = val.cubic)
confusionMatrix(data = val.pred3, val.label)

Confusion Matrix and Statistics

          Reference
Prediction   0   1   2   3   4   5   6   7   8   9
         0 149   0   2   3   0   1   0   0   1   0
         1   0 199   4   2   2   2   5  11  12   3
         2   4  12 157   1   0   1   1   9   8   1
         3   0   2   1 174   0   4   0   3   2   4
         4   3  10   2   0 165   3   9   6   4   9
         5   0   2   1   7   2 141   4   1   5   1
         6   6   0   3   0   8   4 179   0   0   1
         7   0  18   6   6   0   1   0 177   1   2
         8   6   2   7   9   5   5   1   9 144  10
         9   0   4   2   8  13   5   0  12  27 164

Overall Statistics
                                          
               Accuracy : 0.8245          
                 95% CI : (0.8071, 0.8409)
    No Information Rate : 0.1245          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8047          
 Mcnemar's Test P-Value : NA              

Statistics by Class:


Accuracy : 0.8245  

## MODEL4: Bernouli & Stretched Bounding

In [14]:
# model training
model4 <- textmodel_nb(as.dfm(train.cubic), as.factor(train.label), distribution = 'Bernoulli')

In [15]:
# model prediction
val.pred4 <- predict(model4, newdata = as.dfm(val.cubic))
confusionMatrix(data = val.pred4, val.label)

Confusion Matrix and Statistics

          Reference
Prediction   0   1   2   3   4   5   6   7   8   9
         0 152   0   4   1   0   0   1   0   2   0
         1   6 172   4   4   5   9  17  21  35   7
         2   1   5 154   1   0   3   4   3   1   0
         3   1   6   1 185   0  29   0   0   5   4
         4   0   5   0   0 163   2   3   6   4  12
         5   1   4   1   2   1 115  10   3   7   2
         6   6   1   4   0   6   7 164   0   2   1
         7   0   9  10   7   1   0   0 167   1   9
         8   0  35   5   6   5   1   0  12 124  13
         9   1  12   2   4  14   1   0  16  23 147

Overall Statistics
                                          
               Accuracy : 0.7715          
                 95% CI : (0.7525, 0.7897)
    No Information Rate : 0.1245          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7456          
 Mcnemar's Test P-Value : NA              

Statistics by Class:


Accuracy : 0.7715

## Test Four Models & Print 40 Images

Predict based on two untouched models

In [16]:
colnames(dataTest) <- colnames(val.feature) # change Vi to Xi in column names

test.pred1 = predict(model1, newdata = dataTest)
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred1)))
write.csv(output, file = "chaox2_1.csv", row.names = F)

test.pred2 = predict(model2, newdata = dataTest)
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred2)))
write.csv(output, file = "chaox2_3.csv", row.names = F)

Predict based on two stretched bounding models

In [17]:
test.cubic = bounding(dataTest)
colnames(test.cubic) <- colnames(val.feature)[1:400]

test.pred3 = predict(model3, newdata = test.cubic)
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred3)))
write.csv(output, file = "chaox2_2.csv", row.names = F)

test.pred4 = predict(model4, newdata = test.cubic)
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred4)))
write.csv(output, file = "chaox2_4.csv", row.names = F)

print 40 images

In [19]:
for (i in 0:9){
    rows = dataTest[test.pred1 == i, ]
    png(filename=paste(c("Model1",i,".png"), collapse="_"))
    image(matrix(sapply(rows, mean),28,28)[ ,28:1], axes = F, col = gray((0:32)/32))
    dev.off()
}
for (i in 0:9){
    rows = dataTest[test.pred2 == i, ]
    png(filename=paste(c("Model3",i,".png"), collapse="_"))
    image(matrix(sapply(rows, mean),28,28)[ ,28:1], axes = F, col = gray((0:32)/32))
    dev.off()
}
for (i in 0:9){
    rows = dataTest[test.pred3 == i, ]
    png(filename=paste(c("Model2",i,".png"), collapse="_"))
    image(matrix(sapply(rows, mean),28,28)[ ,28:1], axes = F, col = gray((0:32)/32))
    dev.off()
}
for (i in 0:9){
    rows = dataTest[test.pred4 == i, ]
    png(filename=paste(c("Model4",i,".png"), collapse="_"))
    image(matrix(sapply(rows, mean),28,28)[ ,28:1], axes = F, col = gray((0:32)/32))
    dev.off()
}